Je souhaite travailler sur le fichier G_2047.csv pour avoir des statistiques sur l'agriculture française en générale, afin de pouvoir remettre en contexte celles de l'agriculture biologique et de lui donner son juste poids.

J'ai téléchargé la base de données directement sur mon dépôt Git (après maint essais différents pour lire une base de données trop volumineuse) et j'essaie maintenant de la lire

In [1]:
import pandas as pd

# Chemin relatif vers le fichier dans le dépôt Git
FILE_PATH = "./G_2047.csv"

try:
    # Charger le fichier CSV
    data = pd.read_csv(FILE_PATH, sep=";", encoding="utf-8")
    print(f"Données chargées avec succès : {data.shape[0]} lignes et {data.shape[1]} colonnes.")
    print(data.head(3))  # Afficher les 3 premières lignes
except FileNotFoundError:
    print(f"Le fichier {FILE_PATH} est introuvable. Vérifiez que vous êtes dans le bon répertoire.")
except pd.errors.ParserError as e:
    print(f"Erreur de parsing : {e}")


Erreur de parsing : Error tokenizing data. C error: Expected 1 fields in line 3, saw 2



Je ne peux lire la base de données directement car il semblerait qu'il y ait une erreur de séparateur ou autre. Tentons de résoudre cette erreur.

In [2]:
# Lire le fichier brut pour inspecter son contenu
with open("./G_2047.csv", "r", encoding="utf-8") as f:
    content = f.readlines()
    print("Contenu brut du fichier (10 premières lignes) :")
    print("".join(content[:10]))


Contenu brut du fichier (10 premières lignes) :
"Exploitations par tranche de superficie agricole utilisée (SAU)"
""
"Année de référence";"2010"
"France entière";"FR - France entière"
"Champ des exploitations";"Ensemble des exploitations (hors pacages collectifs)"
""
"Source";"Agreste - Recensements"
"notes";"- Les données sont localisées au siège de l'exploitation agricole. Les années 1970 et 1979 ne portent que sur la France métropolitaine."
""
"Superficie agricole utilisée";"Nombre d'exploitations concernées";"Superficie agricole utilisée (ha)";"Volume de travail des exploitations concernées (UTA)";"Production brute standard (millier d'euros)"



Le fichier que j'essaie de lire contient des lignes descriptives en haut (informations générales) qui ne font pas partie des données tabulaires. Ces lignes doivent être ignorées pour charger correctement les données dans Pandas.

En regardant le contenu brut, les données tabulaires semblent commencer à la ligne contenant les colonnes (ici : "Superficie agricole utilisée";"Nombre d'exploitations concernées";...).

Pour résoudre le problème, nous allons ignorer les lignes précédentes (mais nous copions le contenu de ces lignes ci dessous afin de conserver leur contenu).

In [2]:
import pandas as pd

FILE_PATH = "./G_2047.csv"

try:
    # Charger les données en ignorant les 9 premières lignes
    data = pd.read_csv(FILE_PATH, sep=";", encoding="utf-8", skiprows=9)
    print(f"Données chargées avec succès : {data.shape[0]} lignes et {data.shape[1]} colonnes.")
    print(data.head(3))  # Afficher les 3 premières lignes
except pd.errors.ParserError as e:
    print(f"Erreur de parsing : {e}")
except FileNotFoundError:
    print(f"Le fichier {FILE_PATH} est introuvable.")


Données chargées avec succès : 18 lignes et 5 colonnes.
                Superficie agricole utilisée  \
0                                   Ensemble   
1  SAU inférieure à 5 ha, y compris sans SAU   
2                  SAU de 5 à moins de 20 ha   

   Nombre d'exploitations concernées  Superficie agricole utilisée (ha)  \
0                             514694                           27087794   
1                             138723                             267216   
2                              96707                            1049496   

   Volume de travail des exploitations concernées (UTA)  \
0                                             785564      
1                                             117403      
2                                             123422      

   Production brute standard (millier d'euros)  
0                                     52069201  
1                                      5688434  
2                                      5931004  


In [3]:
print(data.columns)

Index(['Superficie agricole utilisée', 'Nombre d'exploitations concernées',
       'Superficie agricole utilisée (ha)',
       'Volume de travail des exploitations concernées (UTA)',
       'Production brute standard (millier d'euros)'],
      dtype='object')


Le document est un tableau à double entrée, j'analyse donc la visualisation disponible sur le site [Agreste](https://agreste.agriculture.gouv.fr/agreste-saiku/?plugin=true&query=query/open/G_2047#query/open/G_2047) pour mieux savoir comment travailler avec. Je remarque que seulement la première ligne m'intéresse puisqu'elle correspond au total des suivantes, qui subdivise les exploitations agricoles par catégorie de surface agricole utilisée.

In [5]:
import pandas as pd

# Charger le fichier CSV
FILE_PATH = "./G_2047.csv"

try:
    # Lire le fichier en ignorant les métadonnées
    data = pd.read_csv(FILE_PATH, sep=";", encoding="utf-8", skiprows=9)

    # Filtrer uniquement la ligne "Ensemble" dans la colonne "Superficie agricole utilisée"
    total_row = data[data.iloc[:, 0] == "Ensemble"]  # Filtre sur la première colonne
    print("Ligne 'Ensemble' extraite avec succès :")
    print(total_row)
    
except FileNotFoundError:
    print(f"Le fichier {FILE_PATH} est introuvable.")
except Exception as e:
    print(f"Erreur : {e}")


Ligne 'Ensemble' extraite avec succès :
  Superficie agricole utilisée  Nombre d'exploitations concernées  \
0                     Ensemble                             514694   

   Superficie agricole utilisée (ha)  \
0                           27087794   

   Volume de travail des exploitations concernées (UTA)  \
0                                             785564      

   Production brute standard (millier d'euros)  
0                                     52069201  


Je sépare maintenant mon tableau en plusieurs tableaux correspondants à chacune des grandes colonnes de mon tableau mère (car chacune de ces colonnes comprend ensuite des sous colonnes en fonction de tous les départements).

In [7]:
# Filtrer uniquement la ligne "Ensemble" dans la colonne "Superficie agricole utilisée"
total_row = data[data.iloc[:, 0] == "Ensemble"]  # Filtre sur la première colonne
print("Ligne 'Ensemble' extraite avec succès :")
print(total_row)

# Sauvegarder la ligne filtrée pour une utilisation ultérieure
total_row.to_csv("filtered_total_row.csv", index=False, sep=";", encoding="utf-8")

Ligne 'Ensemble' extraite avec succès :
  Superficie agricole utilisée  Nombre d'exploitations concernées  \
0                     Ensemble                             514694   

   Superficie agricole utilisée (ha)  \
0                           27087794   

   Volume de travail des exploitations concernées (UTA)  \
0                                             785564      

   Production brute standard (millier d'euros)  
0                                     52069201  


In [11]:
# Charger la ligne filtrée à partir du fichier précédent
total_row = pd.read_csv("filtered_total_row.csv", sep=";", encoding="utf-8")

# Diviser le tableau en sous-tableaux par grandes colonnes
# Sélectionner uniquement la colonne "Superficie agricole utilisée" (sans suffixe "(ha)")
cols_superficie = [col for col in total_row.columns if col.strip() == "Superficie agricole utilisée"]
superficie_table = total_row[cols_superficie]

# Colonnes liées à "Nombre d'exploitations concernées"
cols_exploitations = [col for col in total_row.columns if "Nombre d'exploitations concernées" in col]
exploitations_table = total_row[cols_exploitations]

# Afficher les sous-tableaux sans les sauvegarder
print("\nTableau pour 'Superficie agricole utilisée' :")
print(superficie_table)

print("\nTableau pour 'Nombre d'exploitations concernées' :")
print(exploitations_table)


Tableau pour 'Superficie agricole utilisée' :
  Superficie agricole utilisée
0                     Ensemble

Tableau pour 'Nombre d'exploitations concernées' :
   Nombre d'exploitations concernées
0                             514694
